<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="150px" height="90px" />

## Microestructuras y sistemas de trading

### Proyecto Final.

>##### Emilio de la Torre Jiménez
>##### Carlos Eduardo Ochoa
>##### Diego Velasco
>##### Oscar Villa

### Objetivo:

Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental).

In [2]:
#Importar librerías
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py

In [5]:
#Leemos los archivos indicados
cal=pd.read_csv('calendario_economico.csv', header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True)
cal['timestamp'] = pd.to_datetime(cal['timestamp'])
cal['timestamp'] = cal['timestamp'].dt.tz_localize('UTC')
   
precios=pd.read_csv('precios_historicos_eurusd.csv', header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True)
precios['timestamp'] = pd.to_datetime(precios['timestamp'])
precios['timestamp'] = precios['timestamp'].dt.tz_localize('UTC')

In [6]:
def analisis(evento):
    
    #Filtrar el evento en el Calendario Economico
    tb  = cal[(cal['Name'] == evento) & (cal['Currency'] == 'USD')].reset_index(drop=True).dropna(subset=['actual']).fillna(method='ffill',axis=1)

    #Formar los cuatro grupos; A,B,C,D clasificandolas en el criterio:
    # A Actual >= Consensus >= Previous
    # B Actual >= Consensus < Previous
    # C Actual < Consensus >= Previous
    # D Actual < Consensus < Previous    
    condiciones = [(tb.actual>=tb.consensus) & (tb.consensus>=tb.previous),(tb.actual>=tb.consensus) & (tb.consensus<tb.previous)
                  ,(tb.actual<tb.consensus)&(tb.consensus>=tb.previous),(tb.actual<tb.consensus)&(tb.consensus<tb.previous)]
    elecciones =['a','b','c','d']
    clasification=np.select(condiciones,elecciones)
    tb['clasificacion']=clasification 
    
    #Elimina las fechas del calendario que no esten en el dataFrame de precios.
    tb=tb[tb['timestamp'].isin(list(precios['timestamp']))]
    
    #Crear ventanas de tiempo de media hora antes y media hora después de cada evento del calendario economico.   
    ventanas_tb={}
    for i in range(len(tb)):
        ind=int(precios[precios['timestamp']==tb.iloc[i,0]].index.values.astype(int))
        ventana = precios.iloc[ind-6:ind+7].reset_index(drop=True)
        ventanas_tb[tb.iloc[i,0]]=ventana 
        
    #Calcular las 4 metricas y agregarlos a los Data Frames:
    # (Dirección) Close (t_30) - Open(t_0)
    # (Pips Alcistas) High(t_0 : t_30) – Open(t_0)
    # (Pips Bajistas) Open(t_0) – Low(t_0 : t_30)
    # (Volatilidad) High(t_-30 : t_30) , - mínimo low (t_-30:t_30)    
    d  = []
    pa = []
    pb = []
    v  = []

    for i in range(len(tb)): 
        temp     = ventanas_tb[tb.iloc[i,0]]
        dtemp    = temp.iloc[-1,4] - temp.iloc[6,1]
        pipatemp = max(temp.iloc[6:-1,2]) - temp.iloc[6,1]
        pipbtemp = temp.iloc[6,1] - min(temp.iloc[6:-1,3])
        voltemp  = max(temp.iloc[:,2]) - min(temp.iloc[:,3])

        d.append(dtemp)
        pa.append(pipatemp)
        pb.append(pipbtemp)
        v.append(voltemp)

    tb['d']  = d
    tb['pa'] = pa
    tb['pb'] = pb
    tb['v']  = v

    #codigo para graficar las clasificaciones en barras
    fig = go.Figure()
    fig = make_subplots()
    fig.add_trace(go.Bar(x=list(tb['clasificacion'].value_counts().sort_index().index), 
                         y=tb['clasificacion'].value_counts().sort_index(),name='Conteo',
                             hovertemplate = '<i>Casos</i>: %{y}'
                            '<br><b>Identificador</b>: %{x}<br>'),secondary_y=False)
    # Título del grafico
    fig.update_layout(title_text='Escenarios')

    # Titulo eje x
    fig.update_xaxes(title_text='Escenarios')

    # Titulo eje y
    fig.update_yaxes(title_text='<b>Conteo</b> de escenarios', secondary_y=False)

    grafica=fig    

    return tb, grafica